# LangGraph 中的記憶體最佳化與管理
## 為什麼需要記憶體管理?

當我們建構具備記憶能力的 Agent 時，最容易忽略的就是記憶體管理的重要性。首先是 LLM 的上下文限制問題，不同模型有不同的 token 上限，例如：
- GPT-4 從 8k 到 128k tokens
- Claude 達 200k tokens

但問題在於：Agent 是持續運行的。
當對話與內部狀態不斷累積，很快就會觸及模型的 token 天花板，導致：
- 對話被強制截斷
- 模型注意力被耗散
- token 消耗大
- 效能降低


## 記憶類型與分層概念

要有效管理 Agent 的記憶，第一步是理解：
不是所有記憶都應該被同樣對待。

我們可以依「用途」與「生命週期」將記憶劃分為不同類型。
### 1. 短期對話記憶（Short-term Conversation Memory）
- 用途：維持當前對話的上下文連貫性
- 生命週期：僅限於單一會話
- 常見策略：
    - 只保留最近幾輪
    - 或最近 N 個 tokens

這類記憶不需要永久保存，過期即丟是正常設計。

### 2. 任務與目標記憶（Task / Goal Memory）
- 記錄內容：
    - 當前任務目標
    - 已完成步驟
    - 待辦事項（TODO）

- 特性：與特定任務綁定

當任務完成後，這類記憶可以：
- 被歸檔
- 或直接清除

不需要長期佔用「工作記憶」。

### 3. 長期使用者與知識記憶（Long-term Memory）
這一類包括：
- 使用者偏好
- 累積的背景知識
- 系統運行經驗

理論上它們需要長期保存，但實務上仍必須引入衰減機制，因為：

- 偏好會改變
- 知識會過時
- 長期未使用的記憶，價值會下降

「永久保存」不等於「永久活躍」。

## Active Memory 與 Archived Memory 的分層設計

在實際系統中，一個常見且有效的做法是將記憶分成兩個層級。
### 1. Active Memory（主動記憶）
Active Memory 是系統中的熱資料（Hot Data）層,存放那些頻繁被存取、對當前任務至關重要的記憶。這類記憶通常包括最近的對話歷史、正在進行的任務狀態,以及使用者最常用到的偏好設定。由於需要在每次 Agent 推理時快速讀取，Active Memory 會保存在高效能的儲存系統中，例如記憶體資料庫或向量資料庫的高速索引區。然而，高效能意味著高成本，因此 Active Memory 的容量必須受到嚴格控制，通常只保留數千到數萬條記錄。系統會透過重要性評分、存取頻率和時效性等指標，持續評估哪些記憶應該留在這一層，一旦記憶的價值下降或長時間未被使用，就會被降級到 Archived Memory，為更重要的新記憶騰出空間。

### Archived Memory（封存記憶）
Archived Memory 是系統的冷資料（Cold Data）層，用於長期保存那些不常使用但未來可能還需要的記憶。這些記憶可能是幾週前的對話內容、已完成任務的歷史記錄，或是很少被觸發的使用者偏好。由於存取頻率低，Archived Memory 可以使用成本較低的儲存方案，例如物件儲存或壓縮後的資料庫，檢索速度雖然較慢但容量幾乎不受限制。重要的是,歸檔並不等於刪除,當 Agent 在 Active Memory 中找不到相關資訊時，仍然可以向 Archived Memory 發起查詢，如果發現某條歸檔記憶突然變得重要(例如使用者提到很久以前的話題)，系統可以將其重新啟動並提升回 Active Memory。這種動態的分層機制讓 Agent 既能保持快速反應，又能在需要時調用深層的歷史記憶，就像人類大腦在日常思考時主要依賴短期記憶，但在特定情境下也能喚醒塵封已久的往事。

## 對話上下文壓縮與裁切
### Trimming：限制對話歷史長度

**固定窗口：**

In [ ]:
def fixed_window_trim(messages, max_count=10):
    """保留最近 N 條訊息"""
    return messages[-max_count:]

**滑動窗口：**

In [ ]:
def sliding_window_trim(messages, max_tokens=4000):
    """基於 token 數量的滑動窗口"""
    total_tokens = 0
    trimmed = []

    for msg in reversed(messages):
        msg_tokens = count_tokens(msg)
        if total_tokens + msg_tokens > max_tokens:
            break
        trimmed.insert(0, msg)
        total_tokens += msg_tokens

    return trimmed


**在 LangGraph 中的 reducer 實作**

In [ ]:
from langgraph.graph import MessagesState
from typing import Annotated

def trim_messages_reducer(existing, new):
    """自動裁切訊息的 reducer"""
    all_messages = existing + new
    return sliding_window_trim(all_messages, max_tokens=4000)

class TrimmedState(MessagesState):
    messages: Annotated[list, trim_messages_reducer]

### Summarization：壓縮舊對話

**摘要時機**

- Token 數達閾值(如 80% 上限)
- 每 N 輪對話
- 任務階段切換時

**分段摘要**

- 把完整對話 切成固定大小的區塊

- 每一段獨立摘要

- 最後得到的是「摘要列表（list of summaries）」

In [ ]:
"""
messages = [m1, m2, m3, ..., m60]
   ↓ 
Segment 1 = [m1  ~ m20]
Segment 2 = [m21 ~ m40]
Segment 3 = [m41 ~ m60]
   ↓ 
Summary_1 = summarize(Segment 1)
Summary_2 = summarize(Segment 2)
Summary_3 = summarize(Segment 3)
   ↓ 
summaries = [
  Summary_1,
  Summary_2,
  Summary_3
]
"""

In [ ]:
# 分段摘要:每段獨立總結
def segment_summarize(messages, segment_size=20):
    summaries = []
    for i in range(0, len(messages), segment_size):
        segment = messages[i:i+segment_size]
        segment_text = "\n".join([m.content for m in segment])  # 確保是純文字
        summary = llm.summarize(segment_text)
        summaries.append(summary)
    return summaries

**累積摘要**

- 永遠只有 一份「當前總結」

- 每來新對話，就在舊摘要基礎上滾動更新

In [ ]:
"""
Summary₀
   ↓ + messages₁
Summary₁
   ↓ + messages₂
Summary₂
   ↓ + messages₃
Summary₃
"""

In [ ]:
# 累積摘要:滾動更新總結
def rolling_summarize(current_summary, new_messages):
    # 將 messages 轉成文字
    new_text = "\n".join([m.content for m in new_messages])
    prompt = f"現有摘要:\n{current_summary}\n\n新對話:\n{new_text}\n\n更新摘要:"
    return llm.invoke(prompt)


**Summarization Node 設計**

In [ ]:
def summarization_node(state: dict):
    messages = state["messages"]
    
    total_tokens = count_tokens("\n".join([m.content for m in messages]))
    
    if total_tokens > THRESHOLD:
        # 保留最近 10 條
        old_messages = messages[:-10]
        old_text = "\n".join([m.content for m in old_messages])
        summary = llm.summarize(old_text)
        
        return {
            "messages": [SystemMessage(content=summary)] + messages[-10:],
            "summary_count": state.get("summary_count", 0) + 1
        }
    
    return state


## 記憶價值與生命週期管理

### 記憶生命週期概念

建立 → 使用 → 強化 / 衰減 → 淘汰

### 記憶評分與 Metadata 設計

In [ ]:
import time

class Memory:
    def __init__(self, content: str, tags=None):
        if tags is None:
            tags = []
        self.content = content
        self.metadata = {
            "importance": 0.5,        # 記憶的重要性分數 (0~1)
            "recency": time.time(),    # 最近一次被使用或命中的時間
            "access_count": 0,         # 被存取或引用的次數
            "created_at": time.time(), # 建立時間
            "tags": tags               # 主題或語意標籤
        }

**重要性計算**

In [ ]:
def calculate_importance(memory):
    keywords = ["目標", "錯誤", "決策", "使用者偏好"]
    keyword_hits = sum(1 for kw in keywords if kw in memory.content)
    keyword_score = min(keyword_hits * 0.2, 1.0)  # 保證不超過 1

    # LLM 評估
    llm_score = llm.evaluate_importance(memory.content)  # 0-1

    return min(keyword_score + llm_score, 1.0)


### 記憶衰減機制
**時間衰減**

- 長時間未被使用的記憶，即使曾經重要，也會逐漸失效

- 模擬人類對「過時資訊」的自然遺忘

In [ ]:
def time_decay(memory, decay_rate=0.01):
    """每天衰減 1%"""
    days_elapsed = (now() - memory.metadata["created_at"]).days
    return (1 - decay_rate) ** days_elapsed

**使用頻率衰減**

- 反覆被使用 = 真正有價值

- 防止重要但較舊的記憶被時間衰減過度削弱

In [ ]:
def frequency_boost(memory):
    """存取次數越多越重要"""
    access_count = memory.metadata["access_count"]
    boost = min(math.log(access_count + 1) * 0.1, 0.5)
    return 1.0 + boost

**最近性因素**
- 強化當前上下文相關性
- 提供短期動態權重以補足長期評分模型

In [ ]:
def recency_boost(memory, half_life_days=7):
    """最近存取的記憶獲得加成"""
    days_since_access = (now() - memory.metadata["recency"]).days

    recency_factor = 2 ** (-days_since_access / half_life_days)

    if days_since_access == 0:
        recency_factor = 1.5

    return recency_factor

**混合衰減策略**

In [ ]:
def hybrid_decay(memory):
    """
    混合記憶衰減策略
    - 時間衰減：避免舊記憶永久佔據高權重
    - 使用頻率：反覆被用的記憶更有價值
    - 最近性：短期上下文相關性補償
    """

    # 原始重要度（0 ~ 1）
    base_importance = memory.metadata["importance"]

    # 各種影響因子（皆為倍率）
    time_factor = time_decay(memory)          # < 1
    freq_factor = frequency_boost(memory)     # >= 1
    recency_factor = recency_boost(memory)    # >= 0

    # 混合計算
    final_score = (
        base_importance
        * time_factor
        * freq_factor
        * recency_factor
    )

    # 限制在合理範圍
    memory.metadata["importance"] = max(
        0.0,
        min(final_score, 1.0)
    )


### 定期清理低價值記憶
**閾值策略**

In [ ]:
def cleanup_memories(memories, threshold=0.2):
    """刪除重要性低於閾值的記憶"""
    cleaned = []
    for m in memories:
        importance = m.metadata.get("importance", 0)
        if importance >= threshold:
            cleaned.append(m)
    return cleaned


**批次清理**

In [ ]:
def batch_cleanup(memory_store, batch_size=1000, threshold=0.2):
    """分批清理，避免阻塞，避免修改長度造成錯亂"""
    new_store = []

    for i in range(0, len(memory_store), batch_size):
        batch = memory_store[i:i + batch_size]

        # 更新衰減
        for memory in batch:
            hybrid_decay(memory)

        # 清理後加入新列表
        cleaned_batch = protected_cleanup(batch, threshold)
        new_store.extend(cleaned_batch)

    return new_store


**防止重要記憶誤刪**

In [ ]:
def protected_cleanup(memories, threshold=0.2):
    protected_tags = {"user_preference", "critical_error", "goal"}
    cleaned = []

    for m in memories:
        tags = set(m.metadata.get("tags", []))
        importance = m.metadata.get("importance", 0)

        # 有保護標記直接保留
        if tags & protected_tags:
            cleaned.append(m)
        # 或重要性足夠
        elif importance >= threshold:
            cleaned.append(m)

    return cleaned


## 記憶結構最佳化

### 記憶合併 (Memory Consolidation)
**核心概念**

In [ ]:
"""
當一筆新記憶進來時
→ 找出是否存在「語意相似」的舊記憶
→ 若相似度足夠高，則依策略合併，而非直接新增
"""


**相似度判斷**

In [ ]:
def find_similar_memories(
    new_memory: Memory,
    existing_memories: List[Memory],
    threshold: float = 0.85
):
    similar = []

    for mem in existing_memories:
        similarity = cosine_similarity(
            [new_memory.embedding],
            [mem.embedding]
        )[0][0]

        if similarity >= threshold:
            similar.append((mem, similarity))

    return similar

**合併策略**

In [ ]:
def merge_memories(memories: List[Memory], strategy="summarize") -> Memory:
    if strategy == "replace":
        return max(memories, key=lambda m: m.metadata.get("importance", 0.5))

    if strategy == "merge":
        merged_content = "\n".join(m.content for m in memories)
        return create_memory(
            merged_content,
            importance=max(m.metadata.get("importance", 0.5) for m in memories)
        )

    if strategy == "summarize":
        text = "\n".join(f"- {m.content}" for m in memories)
        prompt = f"請將以下多筆記憶合併成一則精簡且不遺失重點的記憶：\n{text}"

        summary = llm.invoke(prompt).content

        return create_memory(
            summary,
            importance=max(m.metadata.get("importance", 0.5) for m in memories)
        )

    raise ValueError(f"Unknown merge strategy: {strategy}")


## 動態歸檔與分層存儲設計

- Active Memory：高價值、高頻率、低延遲
- Archive Memory：低頻使用、低成本、可再啟動

**何時從 Active 移到 Archive**

In [ ]:
from datetime import datetime, timezone

def should_archive(memory, now=None):
    now = now or datetime.now(timezone.utc)
    meta = memory.metadata

    criteria = [
        meta.get("importance", 0) < 0.3,
        (now - meta.get("last_accessed", now)).days > 30,
        meta.get("access_count", 0) < 2
    ]

    return sum(criteria) >= 2

**歸檔後的檢索策略**

In [ ]:
def retrieve_with_archive(query, active_store, archive_store):
    active_results = active_store.similarity_search_with_score(query, k=5)

    if (
        len(active_results) < 3
        or max(score for _, score in active_results) < 0.7
    ):
        archive_results = archive_store.similarity_search_with_score(query, k=3)

        reactivated = []
        for mem, score in archive_results:
            if score > 0.8:
                reactivate_memory(mem, active_store, archive_store)
                reactivated.append(mem)

        return [m for m, _ in active_results] + reactivated

    return [m for m, _ in active_results]

**LangGraph 中的實作模式**

In [ ]:
def memory_management_node(state: MemoryState):
    active = state["active_memories"]
    archived = state["archived_memories"]

    # 衰減
    for mem in active:
        hybrid_decay(mem)

    archive_flags = {id(m): should_archive(m) for m in active}

    to_archive = [m for m in active if archive_flags[id(m)]]
    active = [m for m in active if not archive_flags[id(m)]]

    archived.extend(to_archive)

    active = consolidate_memories(active)

    return {
        **state,
        "active_memories": active,
        "archived_memories": archived,
    }